In [ ]:
import os
import pandas as pd

# Configure the Groq client
from groq import Groq

# Set your Groq API key
api_key = "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"  # Replace with your actual API key
if not api_key:
    raise ValueError("Please set your Groq API key in the 'api_key' variable.")

client = Groq(api_key=api_key)

# File to read and write
input_file_path = "/content/trials - Copy.csv"  # Input file path
output_file_path = "refined.csv"  # Output file path

# Load the CSV file
df = pd.read_csv(input_file_path, low_memory=False)

# Remove unintended empty rows
df = df.dropna(how="all").reset_index(drop=True)

# Define columns to clean
columns_to_clean = ["Study Title", "Primary Outcome Measures", "Secondary Outcome Measures", "criteria"]

# Merge all columns into a single column for processing
def merge_columns(row):
    return " \n".join([str(row[col]) for col in columns_to_clean if col in row and not pd.isnull(row[col])])

df["Merged_Content"] = df.apply(merge_columns, axis=1)

# Function to clean and extract relationships using Groq
def extract_relationships(content, row_index):
    if pd.isnull(content):
        return [None, None, None, None, None]

    # Truncate content if it exceeds a certain limit to avoid API errors
    max_length = 2000  # Adjust the limit as needed
    if len(content) > max_length:
        content = content[:max_length] + "..."

    

    prompt = (
          "The following text is a merged representation of a clinical trials dataset row. "
          "Extract only valid and concise relationships from it in the exact format below:\n"
          "'Subject', 'Relationship', 'Object' (one per line):\n\n"
          "'involves (Disease)': 'The name of the primary disease or condition being studied "
          '(e.g., "Diabetes", "Hypertension").\'\n\n'
          "'evaluates (Drug)': 'The name of the drug, therapy, or intervention being evaluated. "
          '(e.g., "Aspirin", "Gene Therapy"). \'\n\n'
          "'measures_primary (Primary Outcome)': 'The main outcome or result the trial is designed "
          'to measure, in up to 5 words (e.g., "Reduction in blood pressure").\'\n\n'
          "'measures_secondary (Secondary Outcome)': 'Additional outcomes or results measured in the trial, "
          'in up to 5 words (e.g., "Improved insulin sensitivity").\'\n\n'
          "'has_criteria (Criteria)': 'Key eligibility criteria for participants in up to 5 words "
          '(e.g., "Ages 18-65, no diabetes").\'\n\n'
          "Rules:\n\n"
          "- 'Assume you are a biomedical expert with a deep understanding of clinical trial terminology.'\n"
          # "- 'If the information for an object cannot be found or is ambiguous, write \"None\" instead of guessing.'\n"
          "- 'Objects must be relevant, concise, and meaningful, with a maximum of 5 words. Avoid long phrases, generic terms, or irrelevant outputs.'\n\n"
          "'Content to analyze\n'"
          f"\n{content}"
)

    # Call the Groq API to get a response
    completion = client.chat.completions.create(
        model="gemma2-9b-it",  # Use the appropriate model
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )

    # Extract the response text from the correct object
    response = completion.choices[0].message.content

    # Parse the response text to extract the relationships
    extracted = [None, None, None, None, None]
    for line in response.splitlines():
        if "involves" in line:
            extracted[0] = line.split(",", 2)[-1].strip()
        elif "evaluates" in line:
            extracted[1] = line.split(",", 2)[-1].strip()
        elif "measures_primary" in line:
            extracted[2] = line.split(",", 2)[-1].strip()
        elif "measures_secondary" in line:
            extracted[3] = line.split(",", 2)[-1].strip()
        elif "has_criteria" in line:
            extracted[4] = line.split(",", 2)[-1].strip()

    print(f"Row {row_index} processed.")  # Log row completion
    return extracted

# Ensure output file is cleared or created
with open(output_file_path, 'w') as f:
    f.write("Subject,Relationship,Object\n")

# Process each row, extract relationships, and write to the CSV immediately
for index, row in df.iloc[0:].iterrows():
    extracted = extract_relationships(row["Merged_Content"], index)
    subject = row.get("NCT Number", f"Row {index}")  # Replace "Study ID" with appropriate column name for Subject

    # Write each relationship as a separate row in the output file
    with open(output_file_path, 'a') as f:
        f.write(f"{subject},involves,{extracted[0]}\n")
        f.write(f"{subject},evaluates,{extracted[1]}\n")
        f.write(f"{subject},measures_primary,{extracted[2]}\n")
        f.write(f"{subject},measures_secondary,{extracted[3]}\n")
        f.write(f"{subject},has_criteria,{extracted[4]}\n")

    print(f"Row {index} written to file.")

print(f"Data extraction complete. Updated data saved to: {output_file_path}")
